are 2 vms similar; is there anomaly
changepoint detection
spike - peak detection
multivariate forecasting
regression

train + test

In [ ]:
from nbdev.showdoc import *
 

In [ ]:
import pandas as pd
import csv

In [ ]:
mdir = "/home/sandeep/MaternaDataset/GWA-T-13_Materna-Workload-Traces/Materna-Trace-3/"
#df = pd.read_csv(mdir + "01.csv", quoting=csv.QUOTE_ALL, sep=';')
df = pd.read_csv(mdir + "02.csv", sep = ';', quoting = csv.QUOTE_ALL)

In [ ]:
def set_ts_index(df):
    # convert the column (it's a string) to datetime type
    datetime_series = pd.to_datetime(df['Timestamp'], format='%d.%m.%Y %H:%M:%S', errors='raise')

    # create datetime index passing the datetime series
    datetime_index = pd.DatetimeIndex(datetime_series)

    # assignment is required for index to change (IMP)
    df = df.set_index(datetime_index)
    return df

In [ ]:
dataframes = []
from glob import glob
filenames = glob(mdir + '*.csv')
for idx, f in enumerate(filenames):
    df = pd.read_csv(f, sep=';', quoting = csv.QUOTE_ALL)
    df = set_ts_index(df)
    df.dataframeName = f
    dataframes.append(df)

In [ ]:
print(len(dataframes))
print(dataframes[0].shape)

In [ ]:
df = dataframes[5]

In [ ]:
df.describe()


In [ ]:
df.dtypes

In [ ]:
df.index

In [ ]:
df.keys()

In [ ]:
df = df.drop([ 'Timestamp', 'CPU usage [%]', 'Memory usage [%]'] , axis=1)


In [ ]:
print("Dataframe shape: ", df.shape)
dt = (df.index[-1] - df.index[0])
print(dt.total_seconds()/3600 + 1)


### scaling with robust scaler

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

https://machinelearningmastery.com/robust-scaler-transforms-for-machine-learning/

In [ ]:
from sklearn.preprocessing import robust_scale

mat = robust_scale(df)
df2 = pd.DataFrame(data = mat,  columns=df.columns, index=df.index)
df2.dropna()
df = df2
df

In [ ]:
import matplotlib.dates as mdates

#df2.plot(subplots=True, figsize=(15,6))
with pd.plotting.plot_params.use('x_compat', True):
    df.plot(y=['CPU usage [MHZ]', 'Memory usage [KB]', "Disk read throughput [KB/s]", "Network transmitted throughput [KB/s]", "Disk write throughput [KB/s]", "Network received throughput [KB/s]"], figsize=(15,4))


### correlation

from kaggle
https://www.kaggle.com/kpiyush04/maternaworkloadtraces/notebooks

In [ ]:
import numpy as np

# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotPerColumnDistribution(df, 10, 5)

In [ ]:
df.dataframeName = '6.csv'
plotCorrelationMatrix(df, 10)

In [ ]:
import scipy
plotScatterMatrix(df, 10, 10)

### Grainger causality


In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests


do diff because time series must be stationary

https://stackoverflow.com/questions/51772493/understanding-output-from-statsmodels-grangercausalitytests

In [ ]:
gc_res = grangercausalitytests(data, 10, verbose=True)
print(gc_res)

### T-digest 

pip install

In [ ]:
import tdigest
from tdigest import TDigest

In [ ]:
digest = TDigest()
l = df[['Disk read throughput [KB/s]']].to_numpy().flatten()
print(l.shape)
digest.batch_update(l)

In [ ]:
print(digest.percentile(50))
print(digest.centroids_to_list())

In [ ]:
mean_df = df.rolling(5, win_type='gaussian').sum(std=3)
mean_df.dropna()

In [ ]:
mean_df.describe()

In [ ]:
#df2.plot(subplots=True, figsize=(15,6))
with pd.plotting.plot_params.use('x_compat', True):
    mean_df.plot(y=['CPU usage [MHZ]', 'Memory usage [KB]', "Disk read throughput [KB/s]", "Network transmitted throughput [KB/s]", "Disk write throughput [KB/s]", "Network received throughput [KB/s]"], figsize=(15,4))


### vector arima

https://www.statsmodels.org/stable/examples/notebooks/generated/statespace_varmax.html

In [ ]:
mod = sm.tsa.VARMAX(endog[['dln_inv', 'dln_inc']], order=(0,2), error_cov_type='diagonal')
res = mod.fit(maxiter=1000, disp=False)
print(res.summary())

### multivariate PCA

https://www.statsmodels.org/stable/examples/notebooks/generated/pca_fertility_factors.html